In [10]:
# setup the const variables

# the path of the data
DATA_PATH = (
    "dataset/Image_Classification/dataset_24581896"
)
import glob
maximum_file = [x for x in glob.glob(f"{DATA_PATH}/**/*.jpg")]
print(len(maximum_file))

BATCH_SIZE = 32
STEP_PER_EPOCH = len(maximum_file) // BATCH_SIZE
EPOCH = 50


3236


In [7]:
import tensorflow as tf
import tensorflow.keras.preprocessing as prep



def prepare_data(data_path, batch_size):
    # # create a image generator
    img_gen = prep.image.ImageDataGenerator(
        rescale=1.0 / 255,
        rotation_range=30,
        width_shift_range=0.05,
        height_shift_range=0.05,    
        zoom_range=0.5  ,
        horizontal_flip=True,
        vertical_flip=True,
        validation_split=0.2,
    )

    # load the data
    train_ds = img_gen.flow_from_directory(
        data_path,
        class_mode="categorical",
        target_size=(256, 256),
        seed=24581896,
        subset="training",
        batch_size=batch_size,
    )

    val_ds = img_gen.flow_from_directory(
        data_path,
        class_mode="categorical",
        target_size=(256, 256),
        seed=24581896,
        subset="validation",
        batch_size=batch_size,
    )

    labels = train_ds.class_indices.keys()
    num_classes = len(labels)

    return train_ds, val_ds, labels, num_classes

In [67]:
train_ds, val_ds, class_names, num_classes = prepare_data(DATA_PATH, BATCH_SIZE)

Found 2596 images belonging to 20 classes.
Found 640 images belonging to 20 classes.


In [68]:
type(train_ds)
# train_ds.labels
train_ds.class_indices.keys()

dict_keys(['AMETHYST WOODSTAR', 'BLACK THROATED HUET', 'Bedlington_terrier', 'CAMPO FLICKER', 'DOWNY WOODPECKER', 'ELEGANT TROGON', 'EntleBucher', 'FIERY MINIVET', 'GREATER PRAIRIE CHICKEN', 'Great_Dane', 'KING EIDER', 'KING VULTURE', 'ORNATE HAWK EAGLE', 'Saint_Bernard', 'Shetland_sheepdog', 'Walker_hound', 'black-and-tan_coonhound', 'boxer', 'dingo', 'toy_terrier'])

## step 2. Create a Classifiers mdoel 

1. Customize AlexNet/GoogleNet/ResNet etc. and reduce/increase the layers, Train, and Test for image classification.

In [71]:
import tensorflow
import tensorflow.keras
from tensorflow.keras import models, layers, applications as apps, callbacks
import os

from datetime import datetime

# from models.inception_v4 import inception_v4

# resnet implement


def view_train_chart(history):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(loss))
    import matplotlib.pyplot as plt
    plt.figure()
    plt.plot(epochs, loss, "r", label="Training loss")
    plt.plot(epochs, val_loss, "b", label="Validation loss")
    plt.title("Training and validation loss")
    plt.legend()

def create_model__resnetv2(num_classes):
    # init
    model = models.Sequential(
        [
            layers.Input((256, 256, 3)),
            apps.ResNet152V2(
                include_top=False,
                weights="imagenet",
                pooling="avg",
            ),
            layers.Flatten(),
            layers.Dense(4096, activation="relu"),
            layers.Dense(1000, activation="relu"),
            layers.Dense(num_classes, activation="softmax", name="predictions"),
        ]
    )

    return model


# google net (inception_v4)
def create_model__inception_v3(num_classes):
    model = models.Sequential(
        [
            layers.Input((256, 256, 3)),
            apps.InceptionV3(
                include_top=False,
                weights="imagenet",
                pooling="avg",
            ),
            layers.Flatten(),
            layers.Dense(4096, activation="relu"),
            layers.Dense(1000, activation="relu"),
            layers.Dense(num_classes, activation="softmax", name="predictions"),
        ]
    )
    # include_top = (True,)
    # weights = ("imagenet",)
    # input_tensor = (None,)
    # input_shape = (None,)
    # pooling = (None,)
    # classes = (1000,)
    # classifier_activation = "softmax"
    return model


def train_model(model,model_name, train_ds, val_ds):
    # init

    now_time = f"{datetime.now():%Y-%m-%d_%H-%M-%S}"

    log_file_name = f"./logs/training__{model_name}__{now_time}.csv"

    open(log_file_name, 'a').close()

    model.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics=["acc"],
    )

    ckp_weights_dir = f"./ckp_weights/img_clsr/{model_name}__{now_time}/"
    os.makedirs(ckp_weights_dir, exist_ok=True)

    checkpoint = callbacks.ModelCheckpoint(
        ckp_weights_dir + "{epoch:02d}-{val_loss:.2f}.weights.h5",
        monitor="val_loss",
        verbose=0,
        save_best_only=True,
        save_weights_only=True,
        mode="auto",
        save_freq="epoch",
    )

    cv_logging = callbacks.CSVLogger(log_file_name, separator=",", append=True)
    history = model.fit(
        train_ds,
        steps_per_epoch=STEP_PER_EPOCH,  # 2000 images = batch_size * steps
        epochs=EPOCH,
        validation_data=val_ds,
        validation_steps=STEP_PER_EPOCH//4,  # 1000 images = batch_size * steps
        callbacks=[checkpoint, cv_logging],
    )

    return model, history

In [11]:
# test run, with cpu

# train_ds, val_ds, class_names, num_classes
# with tensorflow.device("/CPU:0"):
def exec_train_ResNet151V2():
    mod_name = "ResNet151V2"
    resnet_mod = create_model__resnetv2(num_classes)
    resnet_mod, hist = train_model(
        resnet_mod, model_name=mod_name, train_ds=train_ds, val_ds=val_ds
    )
    view_train_chart(hist)
    os.makedirs("./models/img_clsr",exist_ok=True)
    resnet_mod.save(f"./models/img_clsr/{mod_name}.keras")

exec_train_ResNet151V2()

NameError: name 'create_model__resnetv2' is not defined

In [3]:
# train_ds, val_ds, class_names, num_classes
# with tensorflow.device("/CPU:0"):
def exec_train_InceptionV3():
    mod_name = "InceptionV3"
    inception_mod = create_model__inception_v3(num_classes)
    inception_mod, hist = train_model(
        inception_mod, model_name=mod_name, train_ds=train_ds, val_ds=val_ds
    )
    view_train_chart(hist)
    os.makedirs("./models/img_clsr", exist_ok=True)
    inception_mod.save(f"./models/img_clsr/{mod_name}.keras")

exec_train_InceptionV3()